# Projeto 1 - módulo 6

## Precificação dinâmica - e-commerce

### Mercari Price Suggestion Challenge - Kaggle

Mercari é um site de revenda de produtos online. Uma dos desafios desse tipo de plataforma é auxiliar o usuário, muitas vezes com pouco conhecimento de vendas, a determinar um preço para os seus produtos de modo a maximizar as chances de venda.

### Sobre este projeto

O presente projeto tem o objetivo de desenvolver um algoritmo que identifique produtos já vendidos similares e sugira ao usuário um preço ótimo para novos produtos cadastrados.


### Preparação do ambiente

Para este projeto, acesse o link https://www.kaggle.com/competitions/mercari-price-suggestion-challenge/overview 


In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score , mean_absolute_error as MAE, mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
base_train = pd.read_csv ('/content/drive/MyDrive/Blue Edtech/data/processed/train_data.csv') 
base_test = pd.read_csv ('/content/drive/MyDrive/Blue Edtech/data/processed/test_data.csv') 

In [ ]:
base_test

,name,category_1,category_2,category_3,item_condition_id,brand_name,price,shipping,item_description,date,stock
0,women s motorcycle tshirt,Women,Tops & Blouses,T-Shirts,3,Harley-Davidson,6.0,1,xl fits like a small,9-4-2018,8
1,alex and ani necklace,Women,Jewelry,Necklaces,2,ALEX AND ANI,11.0,1,path of life alex and ani necklace in gold has...,25-1-2018,8
2,steve madden red lace up heels,Women,Shoes,Pumps,2,Steve Madden,36.0,0,never been worn isn t true to size runs small,30-11-2018,9
3,men s black tuxedo,Men,Suits,Two Button,3,No Brand,54.0,0,worn a few times single pleated pants pants ar...,30-7-2018,7
4,red open back mermaid style dress xs,Women,Dresses,Full-Length,1,No Brand,316.0,0,this full length wine red mermaid style backle...,29-7-2018,34
...,...,...,...,...,...,...,...,...,...,...,...
444494,pink vs sleep shorts,Women,Athletic Apparel,Shorts,3,PINK,12.0,0,size small great condition barely ever worn to...,16-12-2018,7
444495,coach hand bag,Women,Women's Handbags,Hobo,3,Coach,86.0,0,this large black hobo coach is in a very good ...,15-11-2018,3
444496,waffle knit sweater forever,Women,Sweaters,Crewneck,3,FOREVER 21,14.0,0,great condition tags forever american apparel ...,2-3-2018,4
444497,dr dre beats solo,Electronics,"TV, Audio & Surveillance",Headphones,3,Beats,69.0,0,blue fits any head amazing sound no longer in use,19-3-2018,6


In [4]:
# estratificando a amostra para reduzir o viés

base_test = train_test_split(base_test, test_size=1000, random_state=101, stratify= base_test[['category_1', 'item_condition_id']])[1]

In [ ]:
base_test.shape

(1000, 12)

In [5]:
base_train['item_condition_id'] = base_train['item_condition_id'].map({1: 'otimo', 2: 'bom', 3: 'regular', 4: 'ruim', 5: 'pessimo'}, na_action = None)
base_test['item_condition_id'] = base_test['item_condition_id'].map({1: 'otimo', 2: 'bom', 3: 'regular', 4: 'ruim', 5: 'pessimo'}, na_action = None)

In [6]:
base_train['texto'] = base_train['name'] + ' ' + base_train['category_1'] + ' ' + base_train['category_2'] + ' ' + base_train['category_3'] + ' ' + base_train['brand_name'] + ' ' + base_train['item_condition_id'] 
base_test['texto'] = base_test['name'] + ' ' + base_test['category_1'] + ' ' + base_test['category_2'] + ' ' + base_test['category_3'] + ' ' + base_test['brand_name'] + ' ' + base_test['item_condition_id']

In [7]:
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
# extraindo características

textos_train = base_train[['texto']]
stop_words = stopwords.words('english')
tf = TfidfVectorizer(analyzer = 'word', min_df=0, stop_words = stop_words)
tfidf_matrix_train = tf.fit_transform(textos_train['texto'].values.astype('U'))
textos_test = base_test[['texto']]
tfidf_matrix_test = tf.transform(textos_test['texto'].values.astype('U'))
print(tfidf_matrix_train.shape)
tfidf_matrix_test.shape

(1037162, 85247)


(1000, 85247)

In [ ]:
def precificacao_cosseno(produto):

  # calculando a similaridade de coseno do produto selecionado com os demais

  medida = cosine_similarity(tfidf_matrix_test[produto], tfidf_matrix_train)
  select = (base_test.index == produto)
  produto_cadastrado = base_test[select]

# identificando os produtos com coeficiente de similaridade maior ou igual a 0.5

  pos_maximo = np.where(medida[0] >= 0.5 )
  indicacao = base_train.iloc[pos_maximo].price.mean()
  return indicacao

In [ ]:
# # Teste você mesmo!

# produto = int(input('Digite índice do produto: \n'))
# select = (base_test.index == produto)
# produto_cadastrado = base_test[select]
# produto_cadastrado[['name', 'category_1', 'category_2', 'category_3', 'item_description', 'item_condition_id', 'brand_name', 'price']]

In [ ]:
# # lista de produtos similares

# resultado = precificacao_cosseno(produto)
# resultado[['name', 'category_1', 'category_2', 'category_3', 'item_description', 'item_condition_id', 'brand_name', 'price']]

In [ ]:
# preco_min = resultado['price'].min()
# preco_max = resultado['price'].max()
# preco_medio = resultado['price'].mean()
# preco_moda = resultado['price'].mode()
# print(preco_min, preco_max, preco_medio, preco_moda)
# print(produto_cadastrado['price'])

In [ ]:
preco_medio = pd.DataFrame(columns = ['price'])

for produto in range(0,1000):
  resultado = precificacao_cosseno(produto)
  if resultado > 0:
    preco_medio.loc[produto] = [resultado]
  else:
    preco_medio.loc[produto] = [0]

In [ ]:
# calculando as métricas do modelo
print('Root Mean squared erro: %.2f' % np.sqrt(mean_squared_error(base_test['price'], preco_medio['price'])))
print('erro médio absoluto: %.2f' % MAE(base_test['price'], preco_medio['price']))
print('R score: %.2f' % r2_score(base_test['price'], preco_medio['price']))

Root Mean squared erro: 28.16
erro médio absoluto: 17.63
R score: -0.00


In [ ]:
preco_medio['price real'] = y_test['price']

In [ ]:
preco_medio

,price,price real
0,26.170753,6.0
1,22.138868,11.0
2,26.588081,36.0
3,28.133333,54.0
4,36.014493,316.0
...,...,...
9995,53.114583,184.0
9996,20.073655,19.0
9997,50.557522,76.0
9998,25.646833,8.0
